<font size="5">**Battle of the Neighborhoods: Week 2**</font>

In [1]:
# Importing all the libraries that will be needed
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [93]:
#importing Neighborhood data CSV file
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_477046808e4d419cbfad9c17b7417ea6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wPOR7RSCQslmA7bHqtueJGq2-cNyAsiMQM8KWW3XVOwg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_477046808e4d419cbfad9c17b7417ea6.get_object(Bucket='capstoneproject-donotdelete-pr-feibjkxuow1s2x',Key='Neighborhood.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

neighborhoods = pd.read_csv(body)
neighborhoods.head()


,ID,Neighborhood,Community,Population,Latitude,Longitude
0,1,MID CITY,MID CITY,8647,30.436896,-91.122524
1,2,SOUTH BURBANK,SOUTHWEST,13205,30.361173,-91.14829
2,3,GOODWOOD,MID CITY,5078,30.444549,-91.11095
3,4,NORTH BATON ROUGE,NORTH,13947,30.478934,-91.134331
4,5,DELMONT PLACE,NORTH,4877,30.494911,-91.159107


In [94]:
neighborhoods['Latitude'] = neighborhoods['Latitude'].astype(float)
neighborhoods['Longitude'] = neighborhoods['Longitude'].astype(float)

In [95]:
# verifying the amount of communities and neighborhoods in Baton Rouge
print('The dataframe has {} communities and {} neighborhoods.'.format(
        len(neighborhoods['Community'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 9 communities and 45 neighborhoods.


In [96]:
address = 'Baton Rouge, LA'

geolocator = Nominatim(user_agent="br_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Baton Rouge are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Baton Rouge are 30.4459596, -91.18738.


In [21]:
# create map of Baton Rouge using latitude and longitude values
map_br = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, community, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Community'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, community)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_br)  
    
map_br

In [97]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'BZFQDZQG4TMXUFKG0EAD1VRDW5AQT3C1YYN3YJCV5LO5BXS2' # your Foursquare ID
CLIENT_SECRET = 'M43HUPTPRV4TYWGXLARGKM1RF2N50ZQQA35TKJRKJ0AQKYU0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BZFQDZQG4TMXUFKG0EAD1VRDW5AQT3C1YYN3YJCV5LO5BXS2
CLIENT_SECRET:M43HUPTPRV4TYWGXLARGKM1RF2N50ZQQA35TKJRKJ0AQKYU0


In [98]:
# obtaining the function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
# creating a function to obtain nearby venues for all the neighborhoods in Baton Rouge
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#running the above function on each neighborhood and creating a new dataframe called br_venues
LIMIT = 100
br_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

MID CITY
SOUTH CAMPUS
SOUTH BURBANK
GOODWOOD
NORTH BATON ROUGE
DELMONT PLACE
GLEN OAKS / ZION CITY
NORTH SHERWOOD FOREST
SCOTLANDVILLE
BROWNSFIELD
BAKER
CENTRAL
ZACHARY
CORTANA / VILLA DEL REY
BROOKSTOWN
VICTORIA FARMS
FOREST HEIGHTS / SUNNYBROOK
PORT HUDSON
JEFFERSON TERRACE / INNISWOLD
KLEINPETER
BROADMOOR
MONTICELLO
PARK FOREST / OAKCREST
FAIRWOOD
EAST FAIRFIELDS / MELROSE PLACE
SMILEY HEIGHTS / MELROSE EAST
THE AVENUES / SOUTHERN UNIV
COLLEGE TOWN
UNIVERSITY ACRES / WOODSTONE
SOUTH BLUEBONNET / NICHOLSON
KENILWORTH
POLLARD / WOODCHASE
MAYFAIR
WESTMINSTER / PINE PARK
DOWNTOWN EAST
DOWNTOWN
SOUTHSIDE
CITY PARK
VALLEY PARK
BOCAGE / CITIPLACE / CONCORD ESTATES
FAIRFIELDS
EDEN PARK
ISTROUMA / DIXIE
LEGION VILLAGE
JEFFERSON / DRUSILLA
TARA
GOODWOOD HOMESITES
MILLERVILLE
STEVENDALE
O'NEAL
PRIDE / CHANEYVILLE
PERKINS / HIGHLAND
L S U
OLD SOUTH BATON ROUGE
JONES CREEK
JEFFERSON / TIGER BEND
SHENANDOAH
SHERWOOD FOREST


In [100]:
# checking size of the new dataframe
print(br_venues.shape)
br_venues.head()

(253, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MID CITY,30.436896,-91.122524,Hibernia Wealth Management,30.432882,-91.120721,Financial or Legal Service
1,SOUTH BURBANK,30.361173,-91.148290,Hydra-Guard Technologies,30.362677,-91.144547,Food & Drink Shop
2,GOODWOOD,30.444549,-91.110950,BREC's Independence Park,30.444178,-91.106411,Park
3,GOODWOOD,30.444549,-91.110950,Independence Park Tennis Center,30.446632,-91.110444,Tennis Court
4,GOODWOOD,30.444549,-91.110950,Cafeteria,30.447484,-91.108070,Cafeteria


In [101]:
# checking how many venues were returned for each neighborhood
br_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BOCAGE / CITIPLACE / CONCORD ESTATES,45,45,45,45,45,45
BROADMOOR,2,2,2,2,2,2
BROOKSTOWN,1,1,1,1,1,1
BROWNSFIELD,2,2,2,2,2,2
CENTRAL,4,4,4,4,4,4
CITY PARK,8,8,8,8,8,8
CORTANA / VILLA DEL REY,3,3,3,3,3,3
DELMONT PLACE,8,8,8,8,8,8
DOWNTOWN,14,14,14,14,14,14


In [102]:
# how many unique categories can be curated from all the returned venues?
print('There are {} uniques categories.'.format(len(br_venues['Venue Category'].unique())))

There are 119 uniques categories.


In [103]:
# analyzing each neighborhoods
# one hot encoding
br_onehot = pd.get_dummies(br_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
br_onehot['Neighborhood'] = br_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [br_onehot.columns[-1]] + list(br_onehot.columns[:-1])
br_onehot = br_onehot[fixed_columns]

br_onehot.head()

,Yoga Studio,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Station,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,College Football Field,College Library,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Exhibit,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Insurance Office,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Museum,Music Venue,Neighborhood,Office,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Post Office,RV Park,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Thrift / Vintage Store,Video Store,Winery,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,MID CITY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,SOUTH BURBANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GOODWOOD,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GOODWOOD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GOODWOOD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [104]:
# grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
br_grouped = br_onehot.groupby('Neighborhood').mean().reset_index()
br_grouped

,Neighborhood,Yoga Studio,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Station,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,College Football Field,College Library,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Exhibit,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Insurance Office,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Museum,Music Venue,Office,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Post Office,RV Park,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Thrift / Vintage Store,Video Store,Winery,Women's Store
0,BOCAGE / CITIPLACE / CONCORD ESTATES,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.0,0.000000,0.000000,0.022222,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.044444,0.0,0.00,0.022222,0.044444,0.022222,0.044444,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.022222,0.000000,0.000,0.022222,0.000000,0.000000,0.000000,0.0,0.022222,0.000000,0.022222,0.0,0.00,0.000000,0.022222,0.00,0.00,0.00,0.00,0.00,0.000000,0.022222,0.000000,0.044444,0.000,0.000000,0.022222,0.0,0.00,0.000000,0.022222,0.0,0.000000,0.044444,0.022222,0.00,0.022222,0.000000,0.000000,0.0,0.000000,0.044444,0.044444,0.022222,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.022222,0.044444,0.000,0.000000,0.000,0.0,0.000000,0.022222,0.000000,0.022222,0.0,0.022222,0.022222,0.022222,0.044444,0.022222,0.022222,0.000000,0.00,0.022222,0.022222,0.000000,0.000000,0.022222,0.000000,0.000,0.000000,0.00,0.000000,0.022222,0.000000,0.022222
1,BROADMOOR,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.5,0.00,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,BROOKSTOWN,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,1.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000

In [105]:
# print each neighborhood with the 5 most popular venues
num_top_venues = 5

for hood in br_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = br_grouped[br_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BOCAGE / CITIPLACE / CONCORD ESTATES----
                       venue  freq
0         Chinese Restaurant  0.04
1                Pizza Place  0.04
2  Cajun / Creole Restaurant  0.04
3              Grocery Store  0.04
4                 Shoe Store  0.04


----BROADMOOR----
                      venue  freq
0              Home Service   0.5
1              Intersection   0.5
2               Yoga Studio   0.0
3  Mediterranean Restaurant   0.0
4                  Pharmacy   0.0


----BROOKSTOWN----
                    venue  freq
0  Furniture / Home Store   1.0
1             Yoga Studio   0.0
2          Mattress Store   0.0
3                Pharmacy   0.0
4               Pet Store   0.0


----BROWNSFIELD----
                        venue  freq
0  Construction & Landscaping   0.5
1           Other Repair Shop   0.5
2    Mediterranean Restaurant   0.0
3                 Pizza Place   0.0
4                    Pharmacy   0.0


----CENTRAL----
               venue  freq
0  Convenience Store  0.2

                venue  freq
0   Convenience Store  0.33
1  Mexican Restaurant  0.17
2               Hotel  0.17
3       Grocery Store  0.17
4           BBQ Joint  0.17


----VICTORIA FARMS----
                        venue  freq
0  Construction & Landscaping   1.0
1    Mediterranean Restaurant   0.0
2                 Pizza Place   0.0
3                    Pharmacy   0.0
4                   Pet Store   0.0


----WESTMINSTER / PINE PARK----
            venue  freq
0  Breakfast Spot   1.0
1     Yoga Studio   0.0
2            Pool   0.0
3     Pizza Place   0.0
4        Pharmacy   0.0


----ZACHARY----
                venue  freq
0      Cosmetics Shop  0.12
1          Donut Shop  0.12
2  Frozen Yogurt Shop  0.12
3      Sandwich Place  0.12
4          Restaurant  0.12




<font size="5">**Putting that into a Pandas Dataframe**</font>

In [106]:
# writing a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [107]:
# creating the new dataframe and displaying the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = br_grouped['Neighborhood']

for ind in np.arange(br_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(br_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BOCAGE / CITIPLACE / CONCORD ESTATES,Pizza Place,Mediterranean Restaurant,Burger Joint,Cajun / Creole Restaurant,Grocery Store,Shoe Store,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Pharmacy
1,BROADMOOR,Home Service,Intersection,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
2,BROOKSTOWN,Furniture / Home Store,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
3,BROWNSFIELD,Other Repair Shop,Construction & Landscaping,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
4,CENTRAL,Convenience Store,Sandwich Place,Bank,Football Stadium,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store


<font size="5">**Clustering the Neighborhoods**</font>

In [108]:
# Cluster the neighborhoods by running k means and cluser the neighborhoods in 5 clusters
# set number of clusters
kclusters = 5

br_grouped_clustering = br_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(br_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [111]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

br_merged = neighborhoods

# merge br_grouped with br_data to add latitude/longitude for each neighborhood
br_merged = br_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

br_merged.reset_index()

br_merged.head(51) # check the last columns!

,ID,Neighborhood,Community,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,MID CITY,MID CITY,8647,30.436896,-91.122524,1,Financial or Legal Service,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
1,2,SOUTH BURBANK,SOUTHWEST,13205,30.361173,-91.148290,1,Food & Drink Shop,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store
2,3,GOODWOOD,MID CITY,5078,30.444549,-91.110950,1,Park,Tennis Court,Garden,Cafeteria,Women's Store,Financial or Legal Service,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
3,4,NORTH BATON ROUGE,NORTH,13947,30.478934,-91.134331,1,Golf Course,Park,Seafood Restaurant,Women's Store,Financial or Legal Service,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store
4,5,DELMONT PLACE,NORTH,4877,30.494911,-91.159107,1,Gym / Fitness Center,Bank,American Restaurant,Convenience Store,Post Office,Fried Chicken Joint,Bar,Spa,Fast Food Restaurant,Exhibit
5,6,GLEN OAKS / ZION CITY,NORTH,12845,30.517801,-91.136591,0,Park,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
6,7,NORTH SHERWOOD FOREST,EAST,9402,30.455221,-91.053428,1,Thrift / Vintage Store,Discount Store,Bar,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Dog Run
7,8,SCOTLANDVILLE,NORTH,12548,30.534247,-91.164168,1,Rental Car Location,Airport Terminal,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
8,9,BROWNSFIELD,NORTHWEST,7759,30.546435,-91.120714,1,Other Repair Shop,Construction & Landscaping,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
9,10,CENTRAL,NORTHEAST,28040,30.553892,-91.036971,1,Convenience Store,Sandwich Place,Bank,Football Stadium,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store


In [112]:
#Visualizing the Clusters we have created
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(br_merged['Latitude'], br_merged['Longitude'], br_merged['Neighborhood'], br_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<font size="5">**Examining the Clusters**</font>

In [113]:
#cluster 1
br_merged.loc[br_merged['Cluster Labels'] == 0, br_merged.columns[[1] + list(range(5, br_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,GLEN OAKS / ZION CITY,-91.136591,0,Park,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
19,FAIRWOOD,-91.039823,0,Park,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
32,EDEN PARK,-91.155091,0,Park,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop


In [114]:
#cluster 2
br_merged.loc[br_merged['Cluster Labels'] == 1, br_merged.columns[[1] + list(range(5, br_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,MID CITY,-91.122524,1,Financial or Legal Service,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
1,SOUTH BURBANK,-91.148290,1,Food & Drink Shop,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store
2,GOODWOOD,-91.110950,1,Park,Tennis Court,Garden,Cafeteria,Women's Store,Financial or Legal Service,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
3,NORTH BATON ROUGE,-91.134331,1,Golf Course,Park,Seafood Restaurant,Women's Store,Financial or Legal Service,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store
4,DELMONT PLACE,-91.159107,1,Gym / Fitness Center,Bank,American Restaurant,Convenience Store,Post Office,Fried Chicken Joint,Bar,Spa,Fast Food Restaurant,Exhibit
6,NORTH SHERWOOD FOREST,-91.053428,1,Thrift / Vintage Store,Discount Store,Bar,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Dog Run
7,SCOTLANDVILLE,-91.164168,1,Rental Car Location,Airport Terminal,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
8,BROWNSFIELD,-91.120714,1,Other Repair Shop,Construction & Landscaping,Women's Store,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
9,CENTRAL,-91.036971,1,Convenience Store,Sandwich Place,Bank,Football Stadium,Flower Shop,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store
10,ZACHARY,-91.156736,1,Pizza Place,Frozen Yogurt Shop,Donut Shop,Sandwich Place,Restaurant,Pet Store,Asian Restaurant,Cosmetics Shop,Dance Studio,Deli / Bodega


In [115]:
#cluster 3
br_merged.loc[br_merged['Cluster Labels'] == 2, br_merged.columns[[1] + list(range(5, br_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,POLLARD / WOODCHASE,-91.141731,2,Home Service,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop


In [116]:
# Cluster 4
br_merged.loc[br_merged['Cluster Labels'] == 3, br_merged.columns[[1] + list(range(5, br_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,PARK FOREST / OAKCREST,-91.094276,3,Pizza Place,Women's Store,Flower Shop,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
33,ISTROUMA / DIXIE,-91.159844,3,Grocery Store,Pizza Place,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run


In [117]:
#cluster 5
br_merged.loc[br_merged['Cluster Labels'] == 4, br_merged.columns[[1] + list(range(5, br_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,BROADMOOR,-91.067779,4,Home Service,Intersection,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run
21,SMILEY HEIGHTS / MELROSE EAST,-91.130368,4,Intersection,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
34,LEGION VILLAGE,-91.160420,4,Intersection,Women's Store,Food,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop
